In [2]:
import math
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline

In [55]:
df1 = pd.read_csv('raw_datasets_with_pkt_nums/1.tsv', sep='\t', index_col=[0])
df2 = pd.read_csv('raw_datasets_with_pkt_nums/2.tsv', sep='\t', index_col=[0])
df3 = pd.read_csv('raw_datasets_with_pkt_nums/3.tsv', sep='\t', index_col=[0])
df4 = pd.read_csv('raw_datasets_with_pkt_nums/4.tsv', sep='\t', index_col=[0])
df = pd.concat([df1, df2, df3,df4])
df['tcp_dstport'] = df['tcp_dstport'].fillna(-1)
df['tcp_srcport'] = df['tcp_srcport'].fillna(-1)
df = df.astype({'tcp_dstport': 'int32', 
           'tcp_srcport': 'int32'
          })



In [56]:
df = df.reset_index(drop=True)
df

,packet_number,captured_length,eth_dst,eth_src,highest_layer,info_line,interface_captured,ip_dst,ip_src,length,number,project_id,sniff_timestamp,tcp_dstport,tcp_srcport,tcp_stream
0,0,66,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,TCP,8888?34374 [ACK] Seq=1 Ack=1 Win=65535 Len=0 T...,NaN,192.168.1.1,192.168.1.122,66,1,1,2013-06-02 10:19:14,34374,8888,0.0
1,1,74,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=256/1, ttl...",NaN,1.2.3.4,192.168.1.122,74,2,1,2013-06-02 10:19:15,-1,-1,NaN
2,2,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=512/2, ttl...",NaN,82.108.24.168,192.168.1.122,75,3,1,2013-06-02 10:19:19,-1,-1,NaN
3,3,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=768/3, ttl...",NaN,82.108.25.192,192.168.1.122,75,4,1,2013-06-02 10:19:19,-1,-1,NaN
4,4,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=1024/4, tt...",NaN,82.108.6.189,192.168.1.122,75,5,1,2013-06-02 10:19:19,-1,-1,NaN
5,5,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=1280/5, tt...",NaN,82.108.84.10,192.168.1.122,75,6,1,2013-06-02 10:19:19,-1,-1,NaN
6,6,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=1536/6, tt...",NaN,82.108.255.172,192.168.1.122,75,7,1,2013-06-02 10:19:19,-1,-1,NaN
7,7,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=1792/7, tt...",NaN,82.108.62.158,192.168.1.122,75,8,1,2013-06-02 10:19:19,-1,-1,NaN
8,8,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=2048/8, tt...",NaN,82.108.28.12,192.168.1.122,75,9,1,2013-06-02 10:19:19,-1,-1,NaN
9,9,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=2304/9, tt...",NaN,82.108.213.181,192.168.1.122,75,10,1,2013-06-02 10:19:19,-1,-1,NaN


In [58]:
display(df['packet_number'].value_counts()[:10])
display(df['eth_dst'].value_counts()[:10])
display(df['eth_src'].value_counts()[:10])
display(df['highest_layer'].value_counts()[:10])
display(df['info_line'].value_counts()[:10])
display(df['ip_dst'].value_counts()[:10])
display(df['ip_src'].value_counts()[:10])
display(df['length'].value_counts()[:10])
display(df['sniff_timestamp'].value_counts()[:10])
display(df['tcp_stream'].value_counts()[:10])
display(df['tcp_dstport'].value_counts()[:11])
display(df['tcp_srcport'].value_counts()[:11])

0      4
207    4
333    4
60     4
335    4
44     4
319    4
303    4
287    4
28     4
Name: packet_number, dtype: int64

00:26:b9:2b:0b:59    7834
00:23:8b:82:1f:4a    6047
00:16:cb:92:6e:dc    3445
70:5a:b6:51:d7:b2    2104
00:26:22:cb:1e:79    2013
00:0c:29:54:bf:79     814
52:54:00:12:35:02     256
00:30:48:62:4e:4a     195
ff:ff:ff:ff:ff:ff     165
08:00:27:a1:5f:bf     161
Name: eth_dst, dtype: int64

00:0c:29:54:bf:79    7834
00:23:8b:82:1f:4a    7462
70:5a:b6:51:d7:b2    2079
00:16:cb:92:6e:dc    2071
00:26:22:cb:1e:79    2013
00:26:b9:2b:0b:59     814
52:54:00:12:35:00     332
00:08:e2:3b:56:01     195
08:00:27:a1:5f:bf     161
00:30:48:62:4e:4a     153
Name: eth_src, dtype: int64

TCP                15105
ICMP                7705
SOCKS                155
NBNS                 100
HTTP                  88
BROWSER               45
DATA-TEXT-LINES       29
NBSS                  26
DATA                  26
ARP                   24
Name: highest_layer, dtype: int64

Unknown                                                     155
[TCP segment of a reassembled PDU]                          131
Time-to-live exceeded (Time to live exceeded in transit)     38
Destination unreachable (Host unreachable)                   38
HTTP/1.1 304 Not Modified                                    23
Request Announcement 8FD12EDD2DC1462                         20
Browser Election Request                                     16
Registration NB 8FD12EDD2DC1462<20>                          16
Registration NB 8FD12EDD2DC1462<00>                          16
Registration NB WORKGROUP<1e>                                16
Name: info_line, dtype: int64

...

60      9969
75      7599
74      3687
54       646
62       565
1514      83
110       80
66        74
1494      57
1078      53
Name: length, dtype: int64

2010-02-03 01:43:10    4582
2010-02-03 01:34:07    2722
2010-02-03 01:34:08    2085
2010-02-03 01:43:11    1004
2010-02-03 01:34:09     551
2010-02-03 01:43:13     426
2010-02-03 01:34:10     299
2010-02-03 01:43:12     217
2013-06-02 10:19:34      97
2013-06-02 10:20:24      93
Name: sniff_timestamp, dtype: int64

4.0     309
17.0     74
1.0      54
5.0      54
21.0     50
18.0     49
7.0      48
3.0      46
16.0     45
15.0     45
0.0      36
13.0     33
20.0     31
2.0      31
11.0     26
6.0      18
14.0     18
8.0      17
22.0     17
12.0     16
10.0     16
Name: tcp_stream, dtype: int64

-1        7917
 36020    2604
 139       443
 36022     349
 80        258
 1080      184
 445       183
 2152      112
 1108       49
 36021      47
 1117       38
Name: tcp_dstport, dtype: int64

-1        7917
 36020    3002
 36021     396
 36022     349
 139       349
 80        314
 445       161
 2152      159
 1080      135
 1108       37
 1081       29
Name: tcp_srcport, dtype: int64

In [70]:
len(filter_terms)

123

In [80]:
import math
from collections import defaultdict
import re

def add_tokens_to_dict(tokens, tokens_dict, len_df):
    for token in tokens:
        # hack for NaN values - see https://stackoverflow.com/questions/45300367/why-adding-multiple-nan-in-python-dictionary-giving-multiple-entries
        #if pd.isnull(token):
        if not isinstance(token, str) and math.isnan(token):
            token = nan
        tokens_dict[token] += 1 / len_df

def tokenize_line_of_column(line, column):
    tokens = set()

    # add token for the whole line
    #if column != "info_line" or "=" not in line:
    if column != "info_line":

        tokens.add(line)

    if column == "info_line":
        # add token every word between two whitespaces
        for token in line.split():
            token = token.strip(",][)(")
            if "=" not in token and not token.isdigit():  # remove tokens with '='
                tokens.add(token)

        # add [...] tokens
        for token in re.findall('\[.*?\]', line):
            tokens.add(token)

    return tokens

def tokenize_column(df, column):
    """
    Return a list of (token, frequency) pairs sorted in ascending order of frequency.
    Returning also a sorted list of all frequencies.
    :param df:
    :param column:
    :return:
    """
    tokens_dict = defaultdict(float)
    len_df = len(df)

    for idx, line in df[column].iteritems():
        tokens = tokenize_line_of_column(line, column)
        add_tokens_to_dict(tokens, tokens_dict, len_df)

    # sort token_dict by value
    sorted_by_freq_token_frequency_pairs = sorted(tokens_dict.items(), key=lambda kv: kv[1])
    frequencies = [val for token, val in sorted_by_freq_token_frequency_pairs]


    return sorted_by_freq_token_frequency_pairs, frequencies

In [85]:
info_line_terms = [term for term, _ in tokenize_column(df, 'info_line')[0][::-1][:10]]

In [86]:
info_line_terms

['>',
 'SYN',
 '[SYN]',
 'request',
 'Echo',
 'ping',
 'ACK',
 'RST',
 '[RST, ACK]',
 'TCP']

In [87]:
filter_terms = set(list(df['packet_number'].value_counts()[:10].index) +
list(df['eth_dst'].value_counts()[:20].index) +
list(df['eth_src'].value_counts()[:20].index) +
list(df['highest_layer'].value_counts()[:10].index) +
info_line_terms +
list(df['ip_dst'].value_counts()[:20].index) +
list(df['ip_src'].value_counts()[:20].index) +
list(df['length'].value_counts()[:10].index) +
list(df['sniff_timestamp'].value_counts()[:10].index) +
list(df['tcp_stream'].value_counts()[:10].index) +
list(df['tcp_dstport'].value_counts()[:11].index) +
list(df['tcp_srcport'].value_counts()[:11].index))

In [90]:
filter_terms = [str(filter_term) for filter_term in filter_terms]
filter_terms.remove('-1')
filter_terms

['0',
 'DATA',
 '2010-02-03 01:43:10',
 '2010-02-03 01:34:08',
 '4.0',
 '1.0',
 '5.0',
 '7.0',
 '10.0.4.255',
 '3.0',
 '2.0',
 '2010-02-03 01:43:12',
 '11.0',
 '13.0',
 '6.0',
 '15.0',
 '16.0',
 '17.0',
 '18.0',
 '14.0',
 '20.0',
 '21.0',
 '22.0',
 '00:26:b9:2b:0b:59',
 '28',
 '74.125.77.102',
 'ACK',
 'ff:ff:ff:ff:ff:ff',
 '8.0',
 '44',
 '10.0',
 '209.85.227.99',
 '1078',
 '54',
 '10.0.5.15',
 '1080',
 '1081',
 '60',
 '12.0',
 '62',
 '66',
 '00:23:8b:82:1f:4a',
 '74',
 '75',
 '82.108.51.173',
 'TCP',
 '80',
 '1108',
 '192.168.56.52',
 '2010-02-03 01:43:13',
 '1117',
 '2152',
 '110',
 '52:54:00:12:35:02',
 '139',
 '08:00:27:ba:0b:03',
 'HTTP',
 '192.168.56.50',
 '192.150.11.111',
 '[RST, ACK]',
 'NBNS',
 'NBSS',
 '52:54:00:12:35:00',
 '>',
 '01:00:5e:00:00:16',
 '08:00:27:cd:3d:55',
 '36020',
 '36021',
 '36022',
 'ARP',
 '10.0.3.255',
 '10.0.2.255',
 '00:30:48:62:4e:4a',
 '10.0.2.15',
 '192.168.56.51',
 '207',
 '2010-02-03 01:34:09',
 '10.0.4.15',
 '2013-06-02 10:19:34',
 'ICMP',
 '10.

In [91]:
len(filter_terms)

121